In [1]:
# =============================================================================
# PRÁCTICA 5: OPTIMIZACIÓN min|MAX DE FUNCIONES DE n VARIABLES CON RESTRICCIONES
#
# Objetivo:
#     Resolver el problema de optimización Pot3ab, consistente en
#     minimizar y maximizar la función cúbica:
#
#         f(x, y, z) = r*x**3 + s*y**3 + t*z**3
#
#     sujeta a dos restricciones de igualdad:
#
#         r*x + s*y + t*z = a
#         r*x**2 + s*y**2 + t*z**2 = b
#
# Métodos Implementados:
#     1. SOLVER DIRECTO (Punto Único) con Pyomo/Ipopt:
#           - Ejecuta una minimización y una maximización partiendo
#             de puntos iniciales fijos.
#
#     2. SOLVER DIRECTO Multi-Start:
#           - Repite la resolución del problema usando múltiples
#             puntos iniciales aleatorios.
#
#     3. ALGORITMO DEL LAGRANGIANO AUMENTADO (ALM) con Multi-Start:
#           - Transforma el problema con restricciones en una secuencia
#             de problemas sin restricciones mediante multiplicadores
#             de Lagrange y penalización cuadrática, resolviendo cada
#             subproblema con estrategia Multi-Start.
#
# Entradas:
#     - Archivo de entrada: 'Datos5.dat' con r, s, t, a y b.
#
# Salidas:
#     - Archivo de salida: 'Resultados5.sol', donde se escriben todas las
#       soluciones obtenidas por los tres métodos:
# =============================================================================


import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random 
import math 


# ======================== LECTURA Y EXTRACCIÓN DE DATOS ======================

def Leer_Datos(filename):
    
    """
    Lee los coeficientes del archivo de datos.
    """
    
    data = {}
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('|') and '=' in line:
                    try:
                        clean_line = line.replace('|', '').strip()
                        param_part, value_part = clean_line.split('=', 1)
                        param_name = param_part.strip()
                        param_value_str = value_part.strip()
                        if param_name and param_value_str:
                             data[param_name] = float(param_value_str)
                    except Exception:
                        continue
    except FileNotFoundError:
        print(f"Error: Archivo '{filename}' no encontrado.")
        return None
    required_keys = ['r', 's', 't', 'a', 'b']
    if not all(key in data for key in required_keys):
        print(f"Error de Lectura: Faltan claves necesarias en los datos. Encontradas: {list(data.keys())}")
        return None
    return data


# ==================== MÉTODO 1: SOLVER DIRECTO (PUNTO ÚNICO) ====================

def Solver_Pyomo_PU(data):

    """
    Resuelve el problema directamente (min y max) usando IPOPT.
    Utiliza un punto inicial fijo para cada optimización.
    """
    
    results = {
        'xmin': ('N/A', 'N/A', 'N/A'), 'f_min': 'N/A',
        'xmax': ('N/A', 'N/A', 'N/A'), 'f_max': 'N/A'
    }
    
    r, s, t, a, b = data['r'], data['s'], data['t'], data['a'], data['b']
    
    solver = SolverFactory('ipopt')

    # 1. Definición del modelo Pyomo
    model = pyo.ConcreteModel()

    # Variables de optimización con límites
    model.x = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    model.y = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    model.z = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    
    # Restricciones de igualdad
    model.constr_1 = pyo.Constraint(expr=r * model.x + s * model.y + t * model.z == a)
    model.constr_2 = pyo.Constraint(expr=r * model.x**2 + s * model.y**2 + t * model.z**2 == b)

    # Función objetivo
    def obj_rule(model): return r * model.x**3 + s * model.y**3 + t * model.z**3
    model.f = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


    # MINIMIZACIÓN: Arranque desde el origen
    model.f.sense = pyo.minimize
    model.x.value, model.y.value, model.z.value = 0.0, 0.0, 0.0    
    try:
        min_result = solver.solve(model, tee=False)
        if min_result.solver.termination_condition in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
            results['xmin'] = (pyo.value(model.x), pyo.value(model.y), pyo.value(model.z))
            results['f_min'] = pyo.value(model.f)
    except Exception:
        pass

    # MAXIMIZACIÓN: Arranque desde un punto no central
    model.f.sense = pyo.maximize
    model.x.value, model.y.value, model.z.value = 1.0, 0.5, -1.0
    try:
        max_result = solver.solve(model, tee=False)
        if max_result.solver.termination_condition in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
            results['xmax'] = (pyo.value(model.x), pyo.value(model.y), pyo.value(model.z))
            results['f_max'] = pyo.value(model.f)
    except Exception:
        pass

    return results


# ==================== MÉTODO 2: SOLVER DIRECTO (MULTI-START) ====================

def Solver_Pyomo_MS(data, num_starts=10):
    
    """
    Resuelve el problema min y max usando una estrategia 
    Multi-Start: utiliza múltiples puntos iniciales aleatorios para aumentar 
    la probabilidad de encontrar el óptimo global (o el mejor local).
    """
    
    r, s, t, a, b = data['r'], data['s'], data['t'], data['a'], data['b']
    solver = SolverFactory('ipopt')
    
    best_min = float('inf'); best_xmin = ('N/A', 'N/A', 'N/A')
    best_max = float('-inf'); best_xmax = ('N/A', 'N/A', 'N/A')
    
    # Definición del modelo
    model = pyo.ConcreteModel()
    
    model.x = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    model.y = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    model.z = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    
    model.constr_1 = pyo.Constraint(expr=r * model.x + s * model.y + t * model.z == a)
    model.constr_2 = pyo.Constraint(expr=r * model.x**2 + s * model.y**2 + t * model.z**2 == b)
    
    def obj_rule(model): return r * model.x**3 + s * model.y**3 + t * model.z**3
    model.f = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    for i in range(num_starts):
        # Generación de puntos iniciales aleatorios
        x0 = random.uniform(-10.0, 10.0)
        y0 = random.uniform(-10.0, 10.0)
        z0 = random.uniform(-10.0, 10.0)
        
        # --- Minimización ---
        model.f.sense = pyo.minimize
        model.x.value, model.y.value, model.z.value = x0, y0, z0
        try:
            min_result = solver.solve(model, tee=False)
            if min_result.solver.termination_condition in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
                f_current = pyo.value(model.f)
                if f_current < best_min:
                    best_min = f_current
                    best_xmin = (pyo.value(model.x), pyo.value(model.y), pyo.value(model.z))
        except Exception:
            pass

        # --- Maximización ---
        model.f.sense = pyo.maximize
        model.x.value, model.y.value, model.z.value = x0, y0, z0 
        try:
            max_result = solver.solve(model, tee=False)
            if max_result.solver.termination_condition in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
                f_current = pyo.value(model.f)
                if f_current > best_max:
                    best_max = f_current
                    best_xmax = (pyo.value(model.x), pyo.value(model.y), pyo.value(model.z))
        except Exception:
            pass

    return {'xmin': best_xmin, 'f_min': best_min, 'xmax': best_xmax, 'f_max': best_max}


# ==================== MÉTODO  3: LAGRANGIANO AUMENTADO CON MULTI-START ====================

def Construir_Modelo(data, lambda1, lambda2, mu, is_maximization=False):

    """
    Construye el modelo para minimizar el Lagrangiano Aumentado.
    Para la maximización de f(x), se minimiza -f(x).
    """
    
    r, s, t, a, b = data['r'], data['s'], data['t'], data['a'], data['b']
    model = pyo.ConcreteModel()
    model.x = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    model.y = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))
    model.z = pyo.Var(domain=pyo.Reals, bounds=(-10.1, 10.1))

    # Restricciones h(x)
    def h1(model): return r * model.x + s * model.y + t * model.z - a
    def h2(model): return r * model.x**2 + s * model.y**2 + t * model.z**2 - b
    
    # Objetivo original
    def f_x(model): return r * model.x**3 + s * model.y**3 + t * model.z**3
    
    # Lagrangiano aumentado
    def alm_rule(model):
        obj_term = f_x(model) * (-1 if is_maximization else 1)
        lambda_term = lambda1 * h1(model) + lambda2 * h2(model)
        penalty_term = (mu / 2) * (h1(model)**2 + h2(model)**2)
        return obj_term + lambda_term + penalty_term
        
    model.f_alm = pyo.Objective(rule=alm_rule, sense=pyo.minimize)
    
    # Guardamos los valores para leerlos después
    model.h1_val = pyo.Expression(rule=h1)
    model.h2_val = pyo.Expression(rule=h2)
    model.f_orig = pyo.Expression(rule=f_x)

    return model

def Lagrangiano_Aumentado_MS(data, initial_mu, beta, max_iter, tolerance, inner_starts):
    
    """
    Aplica el Algoritmo de Lagrangiano Aumentado. Cada subproblema no restringido
    se resuelve usando una estrategia Multi-Start.
    """
    
    results = {
        'xmin': ('N/A', 'N/A', 'N/A'), 'f_min': 'N/A',
        'xmax': ('N/A', 'N/A', 'N/A'), 'f_max': 'N/A'
    }
    
    solver = SolverFactory('ipopt')
    if not solver.available(): 
        return results

    def run_alm(sense, x0, y0, z0):

        # INICIALIZACIÓN
        x_k, y_k, z_k = x0, y0, z0
        lambda1_k, lambda2_k = 0.0, 0.0 
        mu_k = initial_mu
        is_max = (sense == pyo.maximize)
        
        h_norm_prev = float('inf') 

        for k in range(max_iter):

            # Inicializamos valores por si ningún subproblema mejora
            h1_val = 0.0
            h2_val = 0.0

            best_f_orig = float('inf') if sense == pyo.minimize else float('-inf')
            best_x, best_y, best_z = x_k, y_k, z_k
            
            # =============================
            # SUBPROBLEMAS MULTI-START
            # =============================
            for _ in range(inner_starts):

                # PUNTO INICIAL
                if _ == 0:
                    start_x, start_y, start_z = x_k, y_k, z_k
                else:
                    start_x = random.uniform(-10, 10)
                    start_y = random.uniform(-10, 10)
                    start_z = random.uniform(-10, 10)
                
                model = Construir_Modelo(data, lambda1_k, lambda2_k, mu_k, is_maximization=is_max)
                model.x.value, model.y.value, model.z.value = start_x, start_y, start_z
                
                try:
                    solver.solve(model, tee=False)
                    f_current = pyo.value(model.f_orig)
                    
                    # ELEGIR EL MEJOR
                    if (sense == pyo.minimize and f_current < best_f_orig) or \
                       (sense == pyo.maximize and f_current > best_f_orig):

                        best_f_orig = f_current
                        best_x = pyo.value(model.x)
                        best_y = pyo.value(model.y)
                        best_z = pyo.value(model.z)

                        h1_val = pyo.value(model.h1_val)
                        h2_val = pyo.value(model.h2_val)

                except:
                    continue

            # Si se obtuvo algo válido
            if best_f_orig not in [float('inf'), float('-inf')]:

                x_k, y_k, z_k = best_x, best_y, best_z
                f_x_k = best_f_orig
                
                # NORMA DE VIOLACIÓN
                h_norm_current = math.sqrt(h1_val**2 + h2_val**2)

                # CRITERIO DE PARADA
                if h_norm_current < tolerance:
                    return (x_k, y_k, z_k, f_x_k, k+1)

                lambda1_k = lambda1_k + mu_k * h1_val
                lambda2_k = lambda2_k + mu_k * h2_val

                # ACTUALIZAR MU
                if h_norm_current > 0.25 * h_norm_prev:
                    mu_k *= beta

                h_norm_prev = h_norm_current

            else:
                break

        return (x_k, y_k, z_k, f_x_k, max_iter)


    # ---- MINIMIZACIÓN ----
    x_min, y_min, z_min, f_min, iter_min = run_alm(pyo.minimize, 0.0, 0.0, 0.0)
    results['xmin'] = (x_min, y_min, z_min)
    results['f_min'] = f_min

    # ---- MAXIMIZACIÓN ----
    x_max, y_max, z_max, f_max, iter_max = run_alm(pyo.maximize, 1.0, 0.5, -1.0)
    results['xmax'] = (x_max, y_max, z_max)
    results['f_max'] = f_max

    return results


# ==================== ESCRITURA DE RESULTADOS ====================

def Escribir_Resultados_Completo(resultados_directo, resultados_multi_start, resultados_alm_ms, archivo_salida):
    
    """
    Escribe las soluciones de los tres métodos en el archivo de texto.
    """
    
    def format_block(soluciones, title):
        xmin = soluciones.get('xmin', ('N/A', 'N/A', 'N/A'))
        f_min = soluciones.get('f_min', 'N/A')
        xmax = soluciones.get('xmax', ('N/A', 'N/A', 'N/A'))
        f_max = soluciones.get('f_max', 'N/A')
        
        def format_value(val):
            if isinstance(val, (float, int)):
                return f"{val:<28.15f}"
            return f"{val:^28}"

        output_block = [
            f"--- RESULTADOS MÉTODO: {title} ---\n",
            "=========================================",
            f"|xmin(1) = {format_value(xmin[0])}|",
            f"|xmin(2) = {format_value(xmin[1])}|",
            f"|xmin(3) = {format_value(xmin[2])}|",
            "=========================================",
            "",
            "=========================================",
            f"|f(xmin) = {format_value(f_min)}|",
            "=========================================",
            "",
            "=========================================",
            f"|xmax(1) = {format_value(xmax[0])}|",
            f"|xmax(2) = {format_value(xmax[1])}|",
            f"|xmax(3) = {format_value(xmax[2])}|",
            "=========================================",
            "",
            "=========================================",
            f"|f(xmax) = {format_value(f_max)}|",
            "=========================================",
            "\n"
        ]
        return output_block

    output = []
    output.extend(format_block(resultados_directo, "SOLVER DIRECTO (Punto Único)"))
    output.extend(format_block(resultados_multi_start, "SOLVER DIRECTO (Multi-start)"))
    output.extend(format_block(resultados_alm_ms, "LAGRANGIANO AUMENTADO (Multi-start)"))
    
    with open(archivo_salida, 'w') as f:
        f.write('PRACTICA 5: OPTIMIZACION min|MAX DE FUNCIONES DE n VARIABLES CON RESTRICCIONES \n')
        f.write( '           ==================================================================\n')
        f.write( "\n")
        f.write('\n'.join(output))


# ==================== EJECUTAR PROBLEMA  ======================

fichero_entrada = "Datos5.dat"
fichero_salida = "Solucion5.sol"

num_starts = 10
initial_mu = 10.0
beta = 50.0
max_iter = 100
inner_starts = 10
tol = 1e-6


if __name__ == "__main__":
    
    solver_check_ipopt = SolverFactory('ipopt')
    if not solver_check_ipopt.available():
        print("ERROR: El solver 'ipopt' no está instalado o no se encuentra en el PATH.")
        
    datos = Leer_Datos(fichero_entrada)
    
    if datos:
        # 1. Método Estándar (Punto Único)
        resultados_directo = Solver_Pyomo_PU(datos)
        
        # 2. Método Multi-Start 
        resultados_multi_start = Solver_Pyomo_MS(datos, num_starts)
        
        # 3. Algoritmo de Lagrangiano Aumentado (ALM) con Multi-Start
        resultados_alm_ms = Lagrangiano_Aumentado_MS(datos, initial_mu, beta, max_iter, tol, inner_starts)
        
        # 4. Escribir los resultados combinados
        Escribir_Resultados_Completo(resultados_directo, resultados_multi_start, resultados_alm_ms, fichero_salida)